In [1]:
#importing necessary libaries.
import pandas as pd
import numpy as np

In [2]:
#importing dataset
df_basobas=pd.read_csv('basobas_raw.csv')

In [3]:
df_basobas.head(5)

,price,location,area,facing,no_of_bedroom,no_of_bathroom,no_of_livingroom,no_of_kitchen,parking,road_size,furniture,floors
0,32000000,"Shital height, Imadol",0-3-0-2,East,5.0,4.0,2.0,3.0,4.0,13.0,Semi Furnished,3.0
1,28000000,Shital Height,0-3-0-0,West,5.0,4.0,2.0,3.5,2.0,13.0,Semi Furnished,3.5
2,32500000,House for sale bhangal,NaN,South,NaN,NaN,NaN,NaN,NaN,NaN,Select Furnishing,NaN
3,20000000,Pulchowk birtamode,NaN,East,3.0,1.0,1.0,1.0,3.0,NaN,Select Furnishing,1.0
4,32500000,"Koteshwor, Kathmandu",NaN,West,5.0,3.0,2.0,2.5,1.0,NaN,NaN,2.5


In [4]:
df_basobas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             286 non-null    int64  
 1   location          286 non-null    object 
 2   area              273 non-null    object 
 3   facing            286 non-null    object 
 4   no_of_bedroom     279 non-null    float64
 5   no_of_bathroom    281 non-null    float64
 6   no_of_livingroom  280 non-null    float64
 7   no_of_kitchen     281 non-null    float64
 8   parking           280 non-null    float64
 9   road_size         223 non-null    float64
 10  furniture         285 non-null    object 
 11  floors            281 non-null    float64
dtypes: float64(7), int64(1), object(4)
memory usage: 26.9+ KB


In [5]:
df_basobas['area'].unique()

array(['0-3-0-2', '0-3-0-0', nan, '0-4-0-0', '0-7-0-0', '0-4-2-0',
       '5 aana', '0-5-2-0', '0-4-2-1', '3 aana', '0-6-0-0', '0-4-3-0',
       '0-4-2-2', '8 dhur', '0-3-3-0', '5 ropani', '0-8-0-0', '0-4-0-2',
       '0-4-1-0', '0-2-3-0', '0-4-1-2', '0-8-2-0', '0-5-0-0', '0-3-0-1',
       '0-3-2-0', '0-7-3-0', '0-2-2-0', '0-3-0-3', '0-24-0-0', '0-9-2-0',
       '0-4-1-1', '0-5-2-2', '0-5-1-3', '0-3-1-2', '0-5-3-0', '0-6-1-0',
       '0-12-0-0', '0-5-1-2', '0-3-2-2', '0-10-1-0', '0-8-3-3', 'Nepal',
       '0-8-0-3', '2.5 aana', '0-4-0-1.57', '0-9-0-0', '0-5-1-0',
       '0-3-3-2', '0-3-1-0', '0-13-0-0', '0-18-0-0', '0-19-0-0',
       '1-0-0-0', '0-3-1-3', '3.2 Aana', '4 aana 2 paisa', '0-3-2-1',
       '3.2.3', '5 Anaa', '1955.29', '0-6-2-0', '0-4-1-3', '0-16-0-0',
       '0-3-1-1', '0-10-2-0', '3 aana 2 paisa', '4 ana', '6 aana 3 paisa',
       '0-8-1-0', '0-11-0-0', '0-2-2-3', '0-3-2-3', '0-1-0', '6 aana',
       '4 aana', '4 aana ', '5 aana 1 paisa', '5 aana 2 paisa 2 daam',
       

In [6]:
ropani_to_sqft = 5476
aana_to_sqft = 342.25  # 16 aana in 1 ropani
paisa_to_sqft = 85.56  # 4 paisa in 1 aana
daam_to_sqft = 21.39  # 4 daam in 1 paisa

# Conversion function
def convert_to_sqft(measurement):
    measurement = str(measurement).lower().strip()
    total_sqft = 0

    # Special case: Handle "0-3-3-0" format
    if "-" in measurement:
        parts = measurement.split('-')
        if len(parts) == 4:  # Ensure it has exactly 4 parts
            try:
                ropani, aana, paisa, daam = map(float, parts)
                return (
                    ropani * ropani_to_sqft +
                    aana * aana_to_sqft +
                    paisa * paisa_to_sqft +
                    daam * daam_to_sqft
                )
            except ValueError:
                return None  # Invalid number format

    # General case: Handle "3 anna 5 paisa" and similar formats
    unit_conversion = {
        "ropani": ropani_to_sqft,
        "aana": aana_to_sqft,
        "paisa": paisa_to_sqft,
        "daam": daam_to_sqft,
        "sqft": 1,
        "sq ft": 1,
    }

    parts = measurement.split()
    i = 0
    while i < len(parts):
        try:
            value = float(parts[i])  # Numeric value
            unit = parts[i + 1] if i + 1 < len(parts) else None  # Unit
            if unit in unit_conversion:
                total_sqft += value * unit_conversion[unit]
                i += 2  # Move to the next value-unit pair
            else:
                return None  # Invalid unit
        except (ValueError, IndexError):
            return None  # Invalid format

    return total_sqft

In [7]:
df_basobas['area']=df_basobas['area'].apply(convert_to_sqft)

In [8]:
df_basobas.isnull().sum()

price                0
location             0
area                48
facing               0
no_of_bedroom        7
no_of_bathroom       5
no_of_livingroom     6
no_of_kitchen        5
parking              6
road_size           63
furniture            1
floors               5
dtype: int64

In [9]:
#filling up the less necessay data with their mode.
df_basobas['road_size']=df_basobas['road_size'].fillna(df_basobas['road_size'].mode())
df_basobas['furniture']=df_basobas['furniture'].fillna(df_basobas['furniture'].mode()[0])

In [10]:
#rest of data looks like are more import so nan values will affect our result so dropiing those rows.
df_basobas=df_basobas.dropna()

In [11]:
df_basobas.isnull().sum()

price               0
location            0
area                0
facing              0
no_of_bedroom       0
no_of_bathroom      0
no_of_livingroom    0
no_of_kitchen       0
parking             0
road_size           0
furniture           0
floors              0
dtype: int64

In [12]:
df_basobas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 203 entries, 0 to 278
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             203 non-null    int64  
 1   location          203 non-null    object 
 2   area              203 non-null    float64
 3   facing            203 non-null    object 
 4   no_of_bedroom     203 non-null    float64
 5   no_of_bathroom    203 non-null    float64
 6   no_of_livingroom  203 non-null    float64
 7   no_of_kitchen     203 non-null    float64
 8   parking           203 non-null    float64
 9   road_size         203 non-null    float64
 10  furniture         203 non-null    object 
 11  floors            203 non-null    float64
dtypes: float64(8), int64(1), object(3)
memory usage: 20.6+ KB


In [13]:
df_basobas.head(10)

,price,location,area,facing,no_of_bedroom,no_of_bathroom,no_of_livingroom,no_of_kitchen,parking,road_size,furniture,floors
0,32000000,"Shital height, Imadol",1069.53,East,5.0,4.0,2.0,3.0,4.0,13.0,Semi Furnished,3.0
1,28000000,Shital Height,1026.75,West,5.0,4.0,2.0,3.5,2.0,13.0,Semi Furnished,3.5
5,40000000,"Shital height, Imadol",1369.00,North,6.0,4.0,2.0,2.5,4.0,13.0,Full Furnished,2.5
6,28000000,Bhaisepati,1369.00,South,6.0,3.0,2.0,2.5,4.0,16.0,Full Furnished,2.5
7,40000000,Harisiddhi,2395.75,East,6.0,4.0,2.0,2.5,4.0,13.0,Full Furnished,2.5
8,47000000,"Shital height, Imadol",1540.12,East,6.0,4.0,2.0,3.0,4.0,13.0,Full Furnished,3.0
10,52500000,Bhaisepati,1882.37,East,6.0,5.0,2.0,2.5,4.0,16.0,Full Furnished,2.5
11,42500000,Shital Height,1369.00,North,6.0,4.0,2.0,3.0,4.0,13.0,Full Furnished,3.0
14,63000000,Shital Height,2053.50,South,6.0,5.0,2.0,2.5,4.0,13.0,Full Furnished,2.5
15,28000000,"Gadatantra chowk , Saraswotinagar",1026.75,East,6.0,4.0,2.0,3.0,3.0,14.0,Semi Furnished,3.0


In [17]:
df_basobas.to_csv('preprocessed_data_basobas.csv',index=False)